# Lasso model
9 combinations (sex x featureset)

In [ ]:
import numpy as np
import pandas as pd
import os, re, glob, itertools
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tqdm import tqdm
import logging
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

In [ ]:
LOGGER = logging.getLogger(__name__)
optuna.logging.set_verbosity(optuna.logging.WARN)

# Define Functions

## Define evaluation metrics

In [ ]:
def MAPE(true_y, pred_y):
    return np.mean(np.abs((true_y - pred_y) / true_y)) * 100

loss = make_scorer(MAPE, greater_is_better=False)
LOSS = "MAPE"

## Hyperparameter tuning

In [ ]:
def objective(trial, X, y, folds, scoring, DIR=""):
    alpha = trial.suggest_loguniform("alpha", 1e-8, 1e2)
    reg = Lasso(alpha=alpha, random_state=2021, max_iter=1000)
    cv_results = cross_validate(reg, X, y, cv=folds, return_estimator=True, scoring=scoring, n_jobs=-1)
    score = np.mean(cv_results["test_score"]) * -1
    if DIR:
        joblib.dump(cv_results, os.path.join(DIR, f"model_{trial.number}.pkl"), compress=True)
    return score

def tune_lasso(X, y, folds, scoring, n_trials=50, DIR=""):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X, y, folds, scoring, DIR), n_trials=n_trials, callbacks=[logging_callback])
    return study.best_params

## logging callback

In [ ]:
def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)
    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)
        print(f"Trial {frozen_trial.number} finished with best value: {frozen_trial.value} and parameters: {frozen_trial.params}")

## OOF predictions

In [ ]:
def predict_oof(estimators, X, folds):
    """Calculate Out-of-Fold predictions from trained models."""
    oof_preds = np.zeros_like(X.iloc[:, 0], dtype=float)
    for i, estimator in enumerate(estimators):
        val_index = folds[i][1]
        X_val = X.iloc[val_index]
        oof_preds[val_index] = estimator.predict(X_val)
    return oof_preds

## Feature Importance Calculation

In [ ]:
def calculate_feature_coefficients(model, X, y, folds, scoring):
    cv_results = cross_validate(model, X, y, cv=folds, return_estimator=True, scoring=scoring, n_jobs=-1)
    importances = [est.coef_ for est in cv_results['estimator']]
    mean_importances = np.abs(np.mean(importances, axis=0))
    return mean_importances

## Null importance analysis

In [ ]:
def analyse_null_importance(model, coefficient_func, X, y, folds, scoring, trials=20):
    base_importance = coefficient_func(model, X, y, folds, scoring)
    null_importances = []
    for _ in tqdm(range(trials)):
        y_permuted = np.random.permutation(y).flatten()
        null_importance = coefficient_func(model, X, y_permuted, folds, scoring)
        null_importances.append(null_importance)
    null_importances = np.array(null_importances)
    criterion_percentile = 50
    percentile_null_imp = np.percentile(null_importances, criterion_percentile, axis=0)
    null_imp_score = base_importance / (percentile_null_imp + 1e-6)
    sorted_indices = np.argsort(null_imp_score)[::-1]
    return null_imp_score, sorted_indices

## Feature selection

In [ ]:
def select_features_by_percentage(model, X, y, folds, scoring, null_imp_score, sorted_indices, DIR, use_percentages):
    sorted_columns = X.columns[sorted_indices]
    mean_test_scores = []
    
    for percentage in tqdm(use_percentages):
        num_of_features = int(len(sorted_columns) * percentage / 100)
        if num_of_features == 0:
            continue
        selected_cols = sorted_columns[:num_of_features]
        selected_X = X[selected_cols]
        LOGGER.info(f'Null Importance score TOP {percentage}%')
        LOGGER.info(f'Selected features: {list(selected_cols)}')
        mean_test_score = cv_mean_test_score(model, selected_X, y, folds, scoring)
        LOGGER.info(f'Mean test_score: {mean_test_score}')
        mean_test_scores.append(mean_test_score)
    
    return mean_test_scores

def select_features_by_num_features(model, X, y, folds, scoring, null_imp_score, sorted_indices, DIR, num_features_range):
    sorted_columns = X.columns[sorted_indices]
    selected_features_list = []
    mean_test_scores = []
    
    for num_features in tqdm(num_features_range):
        if num_features == 0:
            continue
        selected_cols = sorted_columns[:num_features]
        selected_X = X[selected_cols]
        LOGGER.info(f'Null Importance score TOP {num_features} features')
        LOGGER.info(f'Selected features: {list(selected_cols)}')
        mean_test_score = cv_mean_test_score(model, selected_X, y, folds, scoring)
        LOGGER.info(f'Mean test_score: {mean_test_score}')
        mean_test_scores.append(mean_test_score)
        selected_features_list.append(selected_cols)
    
    selected_features = selected_features_list[np.argmin(mean_test_scores)]
    joblib.dump(list(selected_features), os.path.join(DIR, "selected_features_by_null_importance.pkl"))
    
    return mean_test_scores, selected_features, selected_features_list

def cv_mean_test_score(model, X, y, folds, scoring):
    cv_results = cross_validate(model, X, y, cv=folds, return_estimator=True, scoring=scoring, n_jobs=-1)
    oof_preds = predict_oof(cv_results['estimator'], X, folds)
    return MAPE(y, oof_preds)

def plot_feature_selection_results(mean_test_scores, DIR):
    
    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax1.plot(range(1, len(mean_test_scores) + 1), mean_test_scores, color='b', label='Mean test score')
    ax1.set_xlabel('Importance TOP n features')
    ax1.set_ylabel('Mean test score (MAPE)')
    
    min_index = np.argmin(mean_test_scores)
    min_value = mean_test_scores[min_index]
    
    ax1.plot(min_index+1, min_value, 'ro')
    ax1.text(min_index+1, min_value, f'features: {min_index+1}, Min: {min_value:.3f}', color='red')
    
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(DIR, "nullimportance_featureselection.pdf"), bbox_inches="tight")
    plt.close()

In [ ]:
def optimize_and_save_best_model(X, y, folds, scoring, best_params, n_trials=200, DIR=""):
    study = optuna.create_study(direction='minimize')
    study.enqueue_trial(best_params)
    study.optimize(lambda trial: objective(trial, X, y, folds, scoring, DIR), n_trials=n_trials, callbacks=[logging_callback])
    
    for file in glob.glob(os.path.join(DIR, "model_*.pkl")):
        i = int(re.findall(r"\d+", file)[-1])
        if i != study.best_trial.number:
            os.remove(file)
        else:
            os.rename(file, os.path.join(DIR, "bestmodel.pkl"))
    
    best_model_cv = joblib.load(os.path.join(DIR, "bestmodel.pkl"))
    return best_model_cv['estimator']

In [ ]:
def plot_regression_results(test_y, y_pred_proba_avg, DIR):
    """Plot regression results."""
    finalMAPE = np.mean(np.abs(test_y.values - y_pred_proba_avg) / test_y.values) * 100
        
    plt.figure()
    plt.scatter(test_y.values, y_pred_proba_avg, label='test_samples',s=1)
    
    plt.xlabel("Chronological Age")
    plt.ylabel("Predicted Age")
    plt.grid(True)
    plt.text(np.min(test_y), np.max(y_pred_proba_avg)-7, '$MAPE =$' + str(finalMAPE.round(3)))    
    plt.savefig(os.path.join(DIR, "pred_vs_true.pdf"))
    plt.close()

# Data Import

In [ ]:
ipt_DIR = "../../../0_data_processing/processed_data/"

In [ ]:
modeltype = "Lasso"

In [ ]:
list_out = ['BOS','BOT', 'RR', 'FR', 'FAI', 'ATI']
list_in = ['ONH_T','ONH_A', 'ONH_V', 'Choroid']
prev_features = [x + "_" + y for x in list_in for y in list_out]

In [ ]:
R = ["0.99"]
SEX = ["male", "female", "both"]
FTYPE = ["prev", "tsfresh", "both"]
all_iter = itertools.product(R, SEX, FTYPE)

use_feature_importance_top_percentages = [100, 75, 50, 40, 30, 25, 20, 15, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
kf = KFold(n_splits=5,random_state=2022,shuffle=True)

## Model Training and Evaluation for All Sex-Feature Combinations

In [ ]:
# Iterate over all combinations of parameters
for r, sextype, featuretype in tqdm(all_iter, total=len(R)*len(SEX)*len(FTYPE)):
    # Load data
    y = pd.read_csv(os.path.join(ipt_DIR, "y.csv"), index_col="group.cmp")
    X = pd.read_csv(os.path.join(ipt_DIR, "X_scaled.csv"), index_col="group.cmp")
    folds_out = joblib.load(os.path.join(ipt_DIR, "indices_5folds.pkl"))
    both_features = joblib.load(os.path.join(ipt_DIR, f"tsfresh_features_var_0_r_{r}.pkl"))
    outDIR = os.path.join("../out", LOSS, r, modeltype, sextype, featuretype)
    os.makedirs(outDIR, exist_ok=True)
    
    # Select features based on feature type
    if featuretype == "prev":
        used_features = prev_features
    elif featuretype == "tsfresh":
        used_features = list(set(both_features) - set(prev_features))
    elif featuretype == "both":
        used_features = both_features
    else:
        raise ValueError("Invalid value for featuretype: {}".format(featuretype))
    
    X = X[used_features]
    y = y.reset_index()
    X = X.reset_index()
    
    # Create a mapping from group.cmp to numeric indices
    str_to_num_mapping = dict(zip(y["group.cmp"], y.index))
    # Update fold indices with numeric indices
    for i in range(len(folds_out)):
        folds_out[i] = (
            [str_to_num_mapping[idx] for idx in folds_out[i][0]],
            [str_to_num_mapping[idx] for idx in folds_out[i][1]]
        )
    
    y = y.drop(columns=["group.cmp"])
    X = X.drop(columns=["group.cmp"])

    # Filter data by sex type    
    if sextype != "both":
        if sextype == "male":
            ind = np.array(y[y["SEX.男1.女0"] == 1].index)
        elif sextype == "female":
            ind = np.array(y[y["SEX.男1.女0"] == 0].index)
        else:
            raise ValueError("Invalid value for sextype: {}".format(sextype))
        
        y = y.loc[ind]
        X = X.loc[ind].reset_index()
        index_mapping = dict(zip(list(X["index"]), list(X.index)))
        X = X.drop(["index"], axis=1)
        y = y.loc[ind].reset_index().drop(["index"], axis=1)
        # Update fold indices with new mapping
        new_folds_out = []
        for train_idx, test_idx in folds_out:
            new_train_idx = [index_mapping[idx] for idx in train_idx if idx in index_mapping]
            new_test_idx = [index_mapping[idx] for idx in test_idx if idx in index_mapping]
            new_folds_out.append((new_train_idx, new_test_idx))
        folds_out = new_folds_out
                
    os.makedirs(os.path.join(outDIR, "optuna"), exist_ok=True)
    
    # Define initial parameters using tune_lasso function
    best_params = tune_lasso(X, y["Age"], folds_out, loss, n_trials=50, DIR="")
    best_alpha = best_params["alpha"]
    
    lasso_reg = Lasso(alpha=best_alpha, random_state=2022, max_iter=1000)
    null_imp_score, sorted_indices = analyse_null_importance(lasso_reg, calculate_feature_coefficients, X, y["Age"], folds_out, loss)

    # First feature selection to reduce calculation
    num_selected_features = sum(null_imp_score > 1)
    # Select columns based on sorted indices and number of selected features
    sorted_columns = X.columns[sorted_indices]
    X = X[sorted_columns].iloc[:, :num_selected_features]
    sorted_indices = np.arange(X.shape[1])
    
    if featuretype == "prev":
        mean_test_scores, selected_features, selected_features_list = select_features_by_num_features(
            lasso_reg, X, y["Age"], folds_out, loss, null_imp_score, sorted_indices, outDIR, num_features_range=range(1, len(sorted_indices) + 1))
    else:
        # Rough survey using use_feature_importance_top_percentages
        mean_test_scores = select_features_by_percentage(
            lasso_reg, X, y["Age"], folds_out, loss, null_imp_score, sorted_indices, outDIR, use_percentages=use_feature_importance_top_percentages)
        
        # Detailed surveys
        best_index = np.argmin(mean_test_scores)
        detailed_percentages_upto = use_feature_importance_top_percentages[max(0, best_index-1)]
        num_features_for_detailed = int(np.ceil(len(sorted_indices) * detailed_percentages_upto / 100))
        
        mean_test_scores, selected_features, selected_features_list = select_features_by_num_features(
            lasso_reg, X, y["Age"], folds_out, loss, null_imp_score, sorted_indices, outDIR, num_features_range=range(1, num_features_for_detailed + 1))
        
    plot_feature_selection_results(mean_test_scores, outDIR)

    # nested CV
    pred_y=np.zeros(y.shape[0])
    for i, f in enumerate(folds_out):
        y_pred_proba_list = []
        tr_ind, te_ind = f[0], f[1]
        tr_x = X.iloc[tr_ind, :].loc[:, selected_features]
        te_x = X.iloc[te_ind, :].loc[:, selected_features]
        tr_y = y.iloc[tr_ind, :]["Age"]
        te_y = y.iloc[te_ind, :]["Age"]
        folds_in = list(kf.split(tr_x, tr_y))
        DIRinloop = os.path.join(outDIR, "optuna", f"outer_{i}")
        os.makedirs(DIRinloop, exist_ok=True)
        best_lasso_models_tmp = optimize_and_save_best_model(tr_x, tr_y, folds_in, loss, best_params, n_trials=200, DIR=DIRinloop)
        for best_model in best_lasso_models_tmp:
            tmp_y_pred = best_model.predict(te_x)
            y_pred_proba_list.append(tmp_y_pred)
        y_pred_proba_avg = np.array(y_pred_proba_list).mean(axis=0)
        pred_y[te_ind] = y_pred_proba_avg        
    
    if sextype == "male":
        pred_y = pred_y[np.array(y[y["SEX.男1.女0"] == 1].index)]
        true_y = y[y["SEX.男1.女0"] == 1].copy()
    elif sextype == "female":
        pred_y = pred_y[np.array(y[y["SEX.男1.女0"] == 0].index)]
        true_y = y[y["SEX.男1.女0"] == 0].copy()
    elif sextype == "both":
        true_y = y.copy()
    
    plot_regression_results(true_y["Age"], pred_y, outDIR)
    true_y["Predicted_age"] = pred_y
    true_y.to_csv(os.path.join(outDIR, "pred_vs_true.csv"))

    # plot coefficitnes
    coefs = []
    for bestDIR in glob.glob(os.path.join(outDIR, "optuna", "**", "bestmodel.pkl"), recursive=True):
        best_lasso_cv = joblib.load(bestDIR)
        best_lasso_models = best_lasso_cv['estimator']
        coefs_tmp = [best_model.coef_ for best_model in best_lasso_models]
        coefs.extend(coefs_tmp)
    
    mean_coefs = np.mean(coefs, axis=0)
    feature_names = X[selected_features].columns
    importance_df = pd.DataFrame(data=coefs, columns=feature_names)
    
    sorted_indices_coef = importance_df.mean(axis=0).abs().sort_values(ascending=False).index
    sorted_importance_df = importance_df.loc[:, sorted_indices_coef]
    
    tops = [10, 20, 50, 200]
    for i in tops:
        PLOT_TOP_N = i
        plot_cols = sorted_importance_df.columns[:PLOT_TOP_N]
        _, ax = plt.subplots(figsize=(8, i*0.3))
        ax.grid()
        ax.set_ylabel('Feature')
        ax.set_xlabel('Coefficients')
        ax.axvline(0, ls='-', color="black", lw=1)
        sns.boxplot(data=sorted_importance_df[plot_cols], boxprops=dict(alpha=.3), orient='h', ax=ax)
        plt.savefig(os.path.join(outDIR, f'feature_coefs_top{PLOT_TOP_N}.pdf'), bbox_inches="tight")
        plt.close()
    
    sorted_importance_df.to_csv(os.path.join(outDIR, 'feature_coefs.csv'))